In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("./data/Datasets_filtering_2.csv")

In [3]:
text_data = data['description'].tolist()

In [4]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(text_data)
sequences = tokenizer.texts_to_sequences(text_data)
max_len = max(len(seq) for seq in sequences)
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_len)

In [5]:
embedding_dim = 100
vocab_size = len(tokenizer.word_index) + 1

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(max_len)
])

In [7]:
#reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr = 0.0001)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics = ['accuracy'])

In [8]:
model.fit(padded_sequences, padded_sequences, epochs=1000)

Epoch 1/1000
9/9 [==============================] - 1s 36ms/step - loss: 173834.0781 - accuracy: 0.0037
Epoch 2/1000
9/9 [==============================] - 0s 36ms/step - loss: 173761.6406 - accuracy: 0.0037
Epoch 3/1000
9/9 [==============================] - 0s 36ms/step - loss: 173384.5938 - accuracy: 0.0037
Epoch 4/1000
9/9 [==============================] - 0s 36ms/step - loss: 171732.0156 - accuracy: 0.0037
Epoch 5/1000
9/9 [==============================] - 0s 37ms/step - loss: 165678.4844 - accuracy: 0.0147
Epoch 6/1000
9/9 [==============================] - 0s 36ms/step - loss: 151132.4844 - accuracy: 0.0366
Epoch 7/1000
9/9 [==============================] - 0s 36ms/step - loss: 139417.9375 - accuracy: 0.0366
Epoch 8/1000
9/9 [==============================] - 0s 36ms/step - loss: 135119.3281 - accuracy: 0.0366
Epoch 9/1000
9/9 [==============================] - 0s 36ms/step - loss: 132419.9375 - accuracy: 0.0366
Epoch 10/1000
9/9 [==============================] - 0s 38ms/ste

In [9]:
embeddings = model.predict(padded_sequences)

9/9 [==============================] - 0s 10ms/step


In [10]:
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

In [11]:
user_input = "tempat yang tenang"
user_sequence = tokenizer.texts_to_sequences([user_input])
user_padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(user_sequence, maxlen=max_len)

In [12]:
user_embedding = model.predict(user_padded_sequence)[0]

1/1 [==============================] - 0s 30ms/step


In [13]:
similarity_scores = []
for embedding in embeddings:
    similarity_scores.append(cosine_similarity(user_embedding, embedding))

In [16]:
top_records_indices = np.argsort(similarity_scores)[-5:]
top_records = data.iloc[top_records_indices]
top_scores = [similarity_scores[i] for i in top_records_indices]

In [17]:
print("Most similar data records:")
#print(top_records)
for i in range(len(top_records)):
    print("Record", i+1)
    print("Similarity Score:", top_scores[i])
    print(top_records.iloc[i])
    print()

Most similar data records:
Record 1
Similarity Score: 0.55839753
_id                                                          129
Kota                                                      Malang
nama                                           Jawa Timur Park 3
description    Taman hiburan ini punya Dino Park, Infinite Wo...
Name: 128, dtype: object

Record 2
Similarity Score: 0.60605514
_id                                                          234
Kota                                                    Surabaya
nama                                            Warkop Pitulikur
description    Tempat nongkrong yang memiliki harga makanan d...
Name: 233, dtype: object

Record 3
Similarity Score: 0.68354386
_id                                                 58
Kota                                            Blitar
nama           Kolam Renang Taman Air Surga (Kranggan)
description    Kolam Renang Taman Air Surga (Kranggan)
Name: 57, dtype: object

Record 4
Similarity Score: 0.70321065
_id 